In [1]:
#Fast feature detection

import cv2
import numpy as np

input_image = cv2.imread('lena.png')
gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
# Version under opencv 3.0.0 cv2.FastFeatureDetector()
fast = cv2.FastFeatureDetector_create()

# Detect keypoints
keypoints = fast.detect(gray_image, None)
print("Number of keypoints with non max suppression:", len(keypoints))
# Draw keypoints on top of the input image
img_keypoints_with_nonmax=input_image.copy()
cv2.drawKeypoints(input_image, keypoints, img_keypoints_with_nonmax, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
cv2.imshow('FAST keypoints - with non max suppression', img_keypoints_with_nonmax)
 
# Disable nonmaxSuppression
fast.setNonmaxSuppression(False)
# Detect keypoints again
keypoints = fast.detect(gray_image, None)
print("Total Keypoints without nonmaxSuppression:", len(keypoints))
# Draw keypoints on top of the input image
img_keypoints_without_nonmax=input_image.copy()
cv2.drawKeypoints(input_image, keypoints, img_keypoints_without_nonmax, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('FAST keypoints - without non max suppression', img_keypoints_without_nonmax)

cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [2]:
# SIFT feature detection
# need rebuild opencv contrib module from source

import cv2
import numpy as np
input_image = cv2.imread('sakura.jpg')
gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
# For version opencv < 3.0.0, use cv2.SIFT()
sift = cv2.xfeatures2d.SIFT_create()
keypoints = sift.detect(gray_image, None)
cv2.drawKeypoints(input_image, keypoints, input_image, keypoints,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('SIFT features', input_image)
cv2.waitKey()

AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'

In [11]:
# template matching
import cv2
import numpy as np

cap = cv2.VideoCapture('WiiPlay.mp4')
# Check if the video file is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open the video file")

# Video Length Cut Parameters 
time_length = 1481.0
fps=30
frame_seq = 4820
no_frame_processed = 0
frame_limit = 5000 - frame_seq
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)


# Mouse Callback function 
mouse_pressed = False
y0 = x0 = w = h = 0
def mouse_callback(event, _x, _y, flags, param):
    global show_img, x0, y0, w, h, mouse_pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_pressed = True
        x0, y0 = _x, _y
        show_img = np.copy(img)
    elif event == cv2.EVENT_MOUSEMOVE:
        if mouse_pressed:
            show_img = np.copy(img)
            cv2.rectangle(show_img, (x0, y0), (_x, _y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_pressed = False
        w, h = _x - x0, _y - y0
cv2.namedWindow('image')
cv2.setMouseCallback('image', mouse_callback)


# Reading first frame for template selection 
ret, frame = cap.read()
height, width = frame.shape[0:2]
frame = cv2.resize(frame, None, fx=0.43, fy=0.43, interpolation=cv2.INTER_AREA)
img = np.copy(frame)
show_img = np.copy(frame)


# Acquire a template through mouse dragging 
while True:
    cv2.imshow('image', show_img)
    k = cv2.waitKey(1)
    if k == ord('a') and not mouse_pressed:
        if w*h > 0:
            break
cv2.destroyAllWindows()


template = np.copy(img[y0:y0+h, x0:x0+w])
print(template.shape)
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
index = 0
#Set default method
method = methods[0]
#Perform template matching with various methods (0~5)
while True:
    ret, frame = cap.read()
    no_frame_processed += 1
    if ret == False:
        break
    if no_frame_processed == frame_limit:
        # choose break or loop video
        #break
        cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
        no_frame_processed = 0
    show_img = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img = np.copy(show_img)

    k = cv2.waitKey(30)
    if k == 27:
        break
    elif k > 0 and chr(k).isdigit():
        index = int(chr(k))
        if 0 <= index < len(methods):
            method = methods[index]
    
    res = cv2.matchTemplate(img, template, eval(method))
    print('min:{} max:{}'.format(np.min(res), np.max(res)))
    res = cv2.normalize(res, None, 0, 1, cv2.NORM_MINMAX)            
    print('min_n:{} max_n:{}'.format(np.min(res), np.max(res)))
    #(minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(result)
            #if index >= methods.index('cv2.TM_SQDIFF'):
            #    loc = np.where(res < 0.2)
            #else:
            #    loc = np.where(res > 0.8)            
            #for pt in zip(*loc[::-1]):
            #    cv2.rectangle(res, (pt[0] - int(w/2), pt[1] - int(h/2)), (pt[0] + int(w/2), pt[1] + int(h/2)),(0, 0, 0), 1)
    res_img = np.copy(res)
    ######################################################
    '''for y in range(0, res.shape[0]):
        for x in range(0, res.shape[1]):
            if (index >= methods.index('cv2.TM_SQDIFF')) and (res[y,x] < 0.0000001):            
                cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(0, 0, 0), 1)
            elif (index < methods.index('cv2.TM_SQDIFF')) and (res[y,x] > 0.9999):            
                cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(0, 0, 0), 1)'''
    ######################################################
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(res)
    
    if index >= methods.index('cv2.TM_SQDIFF'):            
                cv2.rectangle(res_img, (minLoc[0] - int(w/2), minLoc[1] - int(h/2)), 
                              (minLoc[0] + int(w/2), minLoc[1] + int(h/2)),(0, 0, 255), 1)
            
    elif index < methods.index('cv2.TM_SQDIFF'):            
                cv2.rectangle(res_img, (maxLoc[0] - int(w/2), maxLoc[1] - int(h/2)), 
                              (maxLoc[0] + int(w/2), maxLoc[1] + int(h/2)),(0, 0, 255), 1)
    ######################################################
    
    show_img = cv2.resize(img, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    res_img = cv2.resize(res_img, (show_img.shape[1], show_img.shape[0]), fx=1, fy=1)*255
    res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR).astype(np.uint8)
    cv2.putText(res_img, method, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    show_img = np.hstack((show_img, res_img))
    cv2.imshow('tm', show_img)
            
cv2.destroyAllWindows()

(53, 36, 3)
min:-19845088.0 max:19646420.0
min_n:-2.816591404553037e-09 max_n:1.0
min:-20126732.0 max:19646428.0
min_n:-2.2256017473409884e-08 max_n:0.9999999403953552
min:-20577502.0 max:19646428.0
min_n:-1.683614314629267e-08 max_n:1.0
min:-21043220.0 max:19646260.0
min_n:-1.917963743380824e-08 max_n:0.9999999403953552
min:-21288596.0 max:19645628.0
min_n:8.188649758267275e-09 max_n:1.0
min:-21424796.0 max:19645878.0
min_n:-2.458873638033765e-08 max_n:0.9999999403953552
min:-21491402.0 max:19647214.0
min_n:8.535415929600276e-09 max_n:1.0
min:-21413244.0 max:19646086.0
min_n:-1.1483351158858568e-08 max_n:0.9999999403953552
min:-21341520.0 max:19645484.0
min_n:-8.819142749416642e-09 max_n:1.0
min:-21022720.0 max:19644708.0
min_n:6.3300831243395805e-09 max_n:1.0
min:-20598508.0 max:19642658.0
min_n:-1.0212140466592246e-08 max_n:1.0
min:-20340388.0 max:19640864.0
min_n:-2.54981102898455e-08 max_n:0.9999999403953552
min:-20355714.0 max:19640872.0
min_n:3.0524702765433176e-09 max_n:1.0
min

min:-0.45093590021133423 max:0.39887455105781555
min_n:2.2263179744186345e-08 max_n:1.0
min:-0.4405287206172943 max:0.43148136138916016
min_n:-1.121613024679391e-08 max_n:1.0
min:-0.46488359570503235 max:0.42066165804862976
min_n:-2.7229280163965086e-08 max_n:1.0
min:-0.4844878911972046 max:0.39421722292900085
min_n:2.56733869719028e-08 max_n:1.0
min:-0.4918345510959625 max:0.36365750432014465
min_n:9.15297704295881e-09 max_n:1.0
min:-0.5246946215629578 max:0.36183518171310425
min_n:1.865252130528461e-08 max_n:1.0
min:-0.557568371295929 max:0.37151899933815
min_n:2.587147918120536e-08 max_n:1.0
min:-0.5568566918373108 max:0.3734893500804901
min_n:-1.4052588426238799e-08 max_n:1.0
min:-0.5235646963119507 max:0.39466825127601624
min_n:-1.3835759204994247e-08 max_n:1.0
min:-0.4835550785064697 max:0.4129127860069275
min_n:2.4841256163199432e-08 max_n:1.0000001192092896
min:-0.47997668385505676 max:0.4222451448440552
min_n:2.7064537277965428e-08 max_n:1.0
min:-0.4646569788455963 max:0.41192

min:-0.5432788133621216 max:0.5076149702072144
min_n:-1.4767003619908792e-08 max_n:0.9999999403953552
min:-0.5263011455535889 max:0.5076147317886353
min_n:-1.2942962257511681e-08 max_n:1.0
min:-0.5069513916969299 max:0.5077378153800964
min_n:-1.0358704116697481e-08 max_n:1.0
min:-0.48995086550712585 max:0.5077375769615173
min_n:7.745981633888732e-09 max_n:1.0
min:-0.49738332629203796 max:0.5077378153800964
min_n:-1.04835766734368e-08 max_n:1.0
min:-0.5020953416824341 max:0.5077379941940308
min_n:-1.0969003483296547e-08 max_n:0.9999999403953552
min:-0.5076270699501038 max:0.5077379941940308
min_n:-7.404501900509786e-09 max_n:1.0
min:-0.5092966556549072 max:0.5077378153800964
min_n:2.3688414785283385e-08 max_n:1.0
min:-0.5065072178840637 max:0.5077378153800964
min_n:-1.0234945335696466e-08 max_n:1.0
min:-0.5012123584747314 max:0.5077379941940308
min_n:-3.855717523038038e-09 max_n:1.0
min:-0.5191646218299866 max:0.507737398147583
min_n:-1.9246765958769174e-08 max_n:1.0
min:-0.523448228836

min:-0.557568371295929 max:0.37151899933815
min_n:2.587147918120536e-08 max_n:1.0
min:-0.5568566918373108 max:0.3734893500804901
min_n:-1.4052588426238799e-08 max_n:1.0
min:-0.5235646963119507 max:0.39466825127601624
min_n:-1.3835759204994247e-08 max_n:1.0
min:-0.4835550785064697 max:0.4129127860069275
min_n:2.4841256163199432e-08 max_n:1.0000001192092896
min:-0.47997668385505676 max:0.4222451448440552
min_n:2.7064537277965428e-08 max_n:1.0
min:-0.4646569788455963 max:0.41192343831062317
min_n:-3.794525582634378e-09 max_n:1.0
min:-0.45681294798851013 max:0.371882826089859
min_n:-3.16193293770084e-09 max_n:1.0
min:-0.4759896397590637 max:0.369428426027298
min_n:-2.2548633182850608e-08 max_n:1.0
min:-0.4797004461288452 max:0.3643210530281067
min_n:-1.8740934137895238e-08 max_n:1.0
min:-0.5119553208351135 max:0.47340479493141174
min_n:-3.27845128822446e-09 max_n:1.0
min:-0.45770028233528137 max:0.42927268147468567
min_n:-1.794582260572497e-10 max_n:1.0
min:-0.45181015133857727 max:0.45219

min:-0.5020953416824341 max:0.5077379941940308
min_n:-1.0969003483296547e-08 max_n:0.9999999403953552
min:-0.5076270699501038 max:0.5077379941940308
min_n:-7.404501900509786e-09 max_n:1.0
min:-0.5092966556549072 max:0.5077378153800964
min_n:2.3688414785283385e-08 max_n:1.0
min:-0.5065072178840637 max:0.5077378153800964
min_n:-1.0234945335696466e-08 max_n:1.0
min:-0.5012123584747314 max:0.5077379941940308
min_n:-3.855717523038038e-09 max_n:1.0
min:-0.5191646218299866 max:0.507737398147583
min_n:-1.9246765958769174e-08 max_n:1.0
min:-0.5234482288360596 max:0.5077375769615173
min_n:2.0857612526015146e-08 max_n:1.0
min:-0.5248101353645325 max:0.507737398147583
min_n:-1.8404321622256248e-08 max_n:1.0
min:-0.5082712769508362 max:0.5077379941940308
min_n:5.156007176765343e-09 max_n:1.0
min:-0.5019074082374573 max:0.5077375769615173
min_n:1.1939896182866505e-08 max_n:1.0
min:-0.4960183799266815 max:0.5077378153800964
min_n:1.3081393746006142e-09 max_n:1.0
min:-0.4902065694332123 max:0.50773781

...

min:-0.46107959747314453 max:0.4939151108264923
min_n:-1.4270767678681295e-08 max_n:0.9999999403953552
min:-0.43138736486434937 max:0.4829695522785187
min_n:-8.052168709582475e-09 max_n:1.0
min:-0.4175323247909546 max:0.4738669991493225
min_n:5.856549023519619e-09 max_n:1.0
min:-0.4183124303817749 max:0.4780312478542328
min_n:-1.4181637197907548e-08 max_n:1.0
min:-0.43420711159706116 max:0.4746211767196655
min_n:-9.53786383206534e-10 max_n:1.0
min:-0.44872361421585083 max:0.5529075264930725
min_n:4.641055539877925e-09 max_n:1.0
min:-0.4718106985092163 max:0.5169109106063843
min_n:-1.1197286653441552e-08 max_n:0.9999999403953552
min:-0.4853850305080414 max:0.48573702573776245
min_n:-7.209205676872443e-09 max_n:0.9999999403953552
min:-0.49731338024139404 max:0.47763708233833313
min_n:5.621501486530178e-09 max_n:1.0
min:-0.49192407727241516 max:0.45476171374320984
min_n:-3.655070912600422e-09 max_n:1.0
min:-0.4866368770599365 max:0.45459461212158203
min_n:1.94088158878003e-08 max_n:1.0
mi

min:-0.5167995691299438 max:0.31311333179473877
min_n:3.523311420394748e-09 max_n:1.0
min:-0.4931645095348358 max:0.3771532475948334
min_n:-2.7262338164746325e-08 max_n:1.0
min:-0.4517149329185486 max:0.39822426438331604
min_n:5.998124663619819e-09 max_n:1.0
min:-0.4517146944999695 max:0.43924853205680847
min_n:-1.3379150232140091e-08 max_n:0.9999999403953552
min:-0.4517149329185486 max:0.45102164149284363
min_n:2.0424614888270298e-08 max_n:1.0
min:-0.451352596282959 max:0.47041353583335876
min_n:-7.55619566916721e-09 max_n:1.0
min:-0.5090646147727966 max:0.5076712965965271
min_n:1.6928652257774957e-08 max_n:1.0
min:-0.5022242069244385 max:0.5076748728752136
min_n:-1.352709944058006e-08 max_n:1.0
min:-0.5015748143196106 max:0.5076748728752136
min_n:8.738005874420196e-09 max_n:1.0
min:-0.4930638372898102 max:0.5076748728752136
min_n:6.113111794547876e-09 max_n:1.0
min:-0.49783292412757874 max:0.5076739192008972
min_n:-7.12868342134243e-09 max_n:0.9999999403953552
min:-0.4979164302349090

min:-0.4718106985092163 max:0.5169109106063843
min_n:-1.1197286653441552e-08 max_n:0.9999999403953552
min:-0.4853850305080414 max:0.48573702573776245
min_n:-7.209205676872443e-09 max_n:0.9999999403953552
min:-0.49731338024139404 max:0.47763708233833313
min_n:5.621501486530178e-09 max_n:1.0
min:-0.49192407727241516 max:0.45476171374320984
min_n:-3.655070912600422e-09 max_n:1.0
min:-0.4866368770599365 max:0.45459461212158203
min_n:1.94088158878003e-08 max_n:1.0
min:-0.4784795045852661 max:0.45991161465644836
min_n:-2.8347230340841634e-08 max_n:0.9999999403953552
min:-0.4694366455078125 max:0.4003738760948181
min_n:-4.5820343075320125e-09 max_n:1.0
min:-0.4631774127483368 max:0.37156617641448975
min_n:8.700805409489476e-09 max_n:1.0
min:-0.4651714861392975 max:0.37255844473838806
min_n:-2.518705954912548e-08 max_n:0.9999999403953552
min:-0.4905346930027008 max:0.37752199172973633
min_n:5.54069856661954e-09 max_n:1.0
min:-0.48273977637290955 max:0.38267356157302856
min_n:-3.371493306758566

min:-0.5090646147727966 max:0.5076712965965271
min_n:1.6928652257774957e-08 max_n:1.0
min:-0.5022242069244385 max:0.5076748728752136
min_n:-1.352709944058006e-08 max_n:1.0
min:-0.5015748143196106 max:0.5076748728752136
min_n:8.738005874420196e-09 max_n:1.0
min:-0.4930638372898102 max:0.5076748728752136
min_n:6.113111794547876e-09 max_n:1.0
min:-0.49783292412757874 max:0.5076739192008972
min_n:-7.12868342134243e-09 max_n:0.9999999403953552
min:-0.49791643023490906 max:0.5076650381088257
min_n:-1.5606680392465933e-09 max_n:1.0
min:-0.4953846335411072 max:0.507659912109375
min_n:-1.9622774516392383e-09 max_n:1.0
min:-0.5069521069526672 max:0.5077126026153564
min_n:1.3472888582555242e-08 max_n:1.0
min:-0.5209680199623108 max:0.5076865553855896
min_n:2.6587571255731746e-08 max_n:1.0
min:-0.531449019908905 max:0.5077046155929565
min_n:7.185171568835358e-09 max_n:1.0
min:-0.5420733690261841 max:0.5076745748519897
min_n:-2.3239934421326325e-08 max_n:1.0
min:-0.547127902507782 max:0.50765281915

min:-0.4694366455078125 max:0.4003738760948181
min_n:-4.5820343075320125e-09 max_n:1.0
min:-0.4631774127483368 max:0.37156617641448975
min_n:8.700805409489476e-09 max_n:1.0
min:-0.4651714861392975 max:0.37255844473838806
min_n:-2.518705954912548e-08 max_n:0.9999999403953552
min:-0.4905346930027008 max:0.37752199172973633
min_n:5.54069856661954e-09 max_n:1.0
min:-0.48273977637290955 max:0.38267356157302856
min_n:-3.371493306758566e-09 max_n:1.0
min:-0.45474234223365784 max:0.388655424118042
min_n:-1.523311254914006e-09 max_n:1.0
min:-0.45093590021133423 max:0.39887455105781555
min_n:2.2263179744186345e-08 max_n:1.0
min:-0.4405287206172943 max:0.43148136138916016
min_n:-1.121613024679391e-08 max_n:1.0
min:-0.46488359570503235 max:0.42066165804862976
min_n:-2.7229280163965086e-08 max_n:1.0
min:-0.4844878911972046 max:0.39421722292900085
min_n:2.56733869719028e-08 max_n:1.0
min:-0.4918345510959625 max:0.36365750432014465
min_n:9.15297704295881e-09 max_n:1.0
min:-0.5246946215629578 max:0.36

###### %%writefile test
## Practice 9
1. Input images from video file WiiPlay.mp4 with frame number between 4820 and 5000 (i.e., level 15).
2. Acquire a face template from the first frame (frame number = 4820).
3. Use cv2.matchTemplate() to perform template matching on subsequent frames.
4. (Optional) Use cv2.minMaxLoc() to find the best match.
5. Draw a rectangle around the best match and show the output images.
6. Observe which method is better ('cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED') 
7. Upload your Jupyter code file (*.ipynb)